In [2]:
import pennylane as qml
from pennylane.optimize import AdamOptimizer
from sklearn.datasets import load_wine
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [22]:
X, y = load_wine(return_X_y=True)

X = pd.DataFrame(X, columns=['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols',
                             'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue',
                             'OD280/OD315 of diluted wines', 'Proline'])
y = pd.DataFrame(y, columns=['target'])

In [23]:
print(X.var())

Alcohol                             0.659062
Malic acid                          1.248015
Ash                                 0.075265
Alcalinity of ash                  11.152686
Magnesium                         203.989335
Total phenols                       0.391690
Flavanoids                          0.997719
Nonflavanoid phenols                0.015489
Proanthocyanins                     0.327595
Color intensity                     5.374449
Hue                                 0.052245
OD280/OD315 of diluted wines        0.504086
Proline                         99166.717355
dtype: float64


In [24]:
X['Proline_log'] = np.log(X['Proline'])
X.drop('Proline', axis=1, inplace=True)
X['Magnesium_sqrt'] = np.sqrt(X['Magnesium'])
X.drop('Magnesium', axis=1, inplace=True)
print(X.var())

Alcohol                          0.659062
Malic acid                       1.248015
Ash                              0.075265
Alcalinity of ash               11.152686
Total phenols                    0.391690
Flavanoids                       0.997719
Nonflavanoid phenols             0.015489
Proanthocyanins                  0.327595
Color intensity                  5.374449
Hue                              0.052245
OD280/OD315 of diluted wines     0.504086
Proline_log                      0.172314
Magnesium_sqrt                   0.483921
dtype: float64


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [42]:
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [33]:
def data_encoding(x):
    n_qubit = len(x)
    qml.AngleEmbedding(features = x, wires = range(n_qubit) , rotation = "X")
    for i in range(n_qubit):
        if i+1 < n_qubit:
            qml.CNOT(wires=[i, i+1])
        

In [34]:
def classifier(param, x=None):
    data_encoding(x)
    n_layer, n_qubit = param.shape[0], param.shape[1]
    for i in range(n_layer):
        for j in range(n_qubit):
            qml.Rot(param[i, j, 0], param[i, j, 1], param[i, j, 2], wires=j)
        for j in range(n_qubit):
            if j+1 < n_qubit:
                qml.CNOT(wires=[j, j+1])

    return qml.expval(qml.PauliZ(0))

In [37]:
n_qubit = X_train.shape[1]
dev = qml.device('default.qubit', wires=n_qubit)
circuit = qml.QNode(classifier, dev)

In [39]:
def mse_loss(predict, label):
    return np.mean((predict - label)**2)

def cost(param, circuit, X, y):
    exp = []
    for i in range(len(X)):
        pred = circuit(param, x=X[i])
        exp.append(pred)

    return mse_loss(np.array(exp), y)

def accuracy(predicts, labels):
    assert len(predicts) == len(labels)
    return np.sum((np.sign(predicts)*labels+1)/2)/len(predicts)

In [45]:
lr = 0.01
opt = AdamOptimizer(lr)
batch_size = 4

n_epoch = 50
cost_train, cost_test, acc_train, acc_test = [], [], [], []

param = np.random.rand(2, n_qubit, 3)
param = param*2*np.pi

for i in range(n_epoch):
    index = np.random.permutation(len(X_train))
    feat_train, label_train = X_train[index], y_train[index]

    for j in range(0, len(X_train), batch_size):
        feat_train_batch = feat_train[j*batch_size:(j+1)*batch_size]
        label_train_batch = label_train[j*batch_size:(j+1)*batch_size]

        param = opt.step(lambda v: cost(v, circuit, feat_train_batch, label_train_batch), param)

    cost_train.append(cost(param, circuit, X_train, y_train))
    cost_test.append(cost(param, circuit, X_test, y_test))

    pred_train = []
    for j in range(len(X_train)):
        pred_train.append(circuit(param, x=X_train[j]))

    acc_train.append(accuracy(np.array(pred_train), y_train))

    pred_test = []
    for j in range(len(X_test)):
        pred_test.append(circuit(param, x=X_test[j]))
        
    acc_test.append(accuracy(np.array(pred_test), y_test))

c:\Users\Abdo\Anaconda3\envs\Quantum\lib\site-packages\pennylane\_grad.py:157: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
c:\Users\Abdo\Anaconda3\envs\Quantum\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Abdo\Anaconda3\envs\Quantum\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
epochs = np.arange(n_epoch) + 1
plt.plot(epochs, cost_train, label='Training Cost',
marker='o')
plt.plot(epochs, cost_test, label='Test Cost', marker='o')
plt.title('Cost Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Cost')
plt.legend()
plt.grid()

# Plot training and test accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, acc_train, label='Training Accuracy',
marker='o')
plt.plot(epochs, acc_test, label='Test Accuracy', marker='o')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()